In [1]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd

In [2]:
# Indicators functions

# SMA :- Simple Moving Average.
def SMA(col: pd.Series, period: int = 50):
    """
    Calculate SMA using given column
    col : pd.Series
    period : int (default)
    """
    return col.rolling(period).mean().round(2)

# RSI :- Relative Strength Index.
def RSI(change : pd.Series, period: int = 14):
    """
    calculate RSI using price change data.
    change : pd.Sereies
    period : int
    """
    gain = change.where(change > 0, 0)
    loss = change.where(change < 0 , 0)
    loss = loss.abs()
    avgain = gain.rolling(period).mean()
    avloss = loss.rolling(period).mean()
    rs = avgain / avloss
    rsi = 100 - (100 / (1 + rs))
    return rsi.round(2)

# ADX :- Average Directional Index
def ADX(high, low, close, period:int = 14):
    TR = pd.DataFrame({
        'a': high - low,
        'b': abs(high - close.shift()),
        'c': abs(low - close.shift())
    }).max(axis=1)

    # Calculate the Directional Movement (DM)
    DMplus = (high - high.shift()) > (low.shift() - low)
    DMminus = (low.shift() - low) > (high - high.shift())

    # Calculate the True Directional Indicator (+DI) and (-DI)
    TR14 = TR.rolling(window=period).sum()
    DM14plus = DMplus.rolling(window=period).sum()
    DM14minus = DMminus.rolling(window=period).sum()

    DIplus14 = DM14plus / TR14 * 100
    DIminus14 = DM14minus / TR14 * 100

    # Calculate the Directional Movement Index (DMI)
    DX14 = abs(DIplus14 - DIminus14) / (DIplus14 + DIminus14) * 100
    ADX14 = DX14.rolling(window=period).mean()
    return ADX14.round(2)


# calcualte volume weigthed average price.
def VWAP(price: pd.Series, vol:pd.Series, period:int = 14) -> pd.Series:
    num = price * vol
    vmp = num.rolling(period).sum() 
    sv = vol.rolling(period).sum()
    return vmp/sv

# calcualting macd.
def MACD():
    pass

In [3]:
# reading smallcap file
smcap = pd.read_csv("nifty500.csv", names= ['symbol', 'open', 'high', 'low', 'close' , 'ltp', 'chng', '%chng', 'volume', 'value', '52w H' , '52w L', '365d % chng' , '30 d % chng'])
smcap = smcap.iloc[2:]
smcap

,symbol,open,high,low,close,ltp,chng,%chng,volume,value,52w H,52w L,365d % chng,30 d % chng
2,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AIA Engineering Ltd.,Capital Goods,AIAENG,EQ,INE212H01026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,APL Apollo Tubes Ltd.,Capital Goods,APLAPOLLO,EQ,INE702C01027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,Zomato Ltd.,Consumer Services,ZOMATO,EQ,INE758T01015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,Zydus Lifesciences Ltd.,Healthcare,ZYDUSLIFE,EQ,INE010B01027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# gething stocks which are close  (under 5 % above )  to 20 d MA.
tv = TvDatafeed('mohitrathor7729', 'Mohit7729@#')
for sym in smcap['symbol']:
#     try :
    data = tv.get_hist(symbol = sym, exchange='NSE', interval= Interval.in_daily, n_bars = 1000)
    print(data['high'])
    data['ADX'] = ADX( data['high'], data['low'], data['close'])
    data['SMA20'] = SMA(data['close'], period = 20)
    print(sym)
    if  -5 < ((data['close'].iloc[-1] / data['SMA20'].iloc[-1])*100) < 5 :
        print(list(data.iloc[-1]))       
#     except Exception as e:
#         print(sym, 'exception : ', e)

ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


TypeError: 'NoneType' object is not subscriptable

In [18]:
# gething stocks which are close  (under 10 % above )  to 200 d MA.
tv = TvDatafeed('mohitrathor7729', 'Mohit7729@#')
for sym in smcap['symbol']:
    try :
        data = tv.get_hist(symbol = sym, exchange='NSE', interval= Interval.in_daily, n_bars = 1000)
        data['ADX'] = ADX(data['high'], data['low'], data['close'])
        data['SMA200'] = SMA(data['close'], period = 20)
        print(sym)
        if data['close'].iloc[-1] > data['SMA200'].iloc[-1] and ((data['close'].iloc[-1] / data['SMA200'].iloc[-1])*100) < 10 :
            print(list(data.iloc[-1]))       
    except Exception as e:
        print(sym, 'exception : ', e)

,symbol,open,high,low,close,volume
datetime,,,,,,
2021-12-23 09:15:00,NSE:MEDPLUS,1040.00000,1143.90000,1040.00000,1121.15000,18407607.0
2021-12-24 09:15:00,NSE:MEDPLUS,1107.00000,1124.60000,1026.20000,1076.05000,4055492.0
2021-12-27 09:15:00,NSE:MEDPLUS,1050.00000,1067.90000,1025.30000,1037.25000,622621.0
2021-12-28 09:15:00,NSE:MEDPLUS,1040.00000,1047.50000,997.00000,1002.65000,556747.0
2021-12-29 09:15:00,NSE:MEDPLUS,1000.00000,1100.00000,995.09998,1075.95000,1378582.0
...,...,...,...,...,...,...
2023-06-12 09:15:00,NSE:MEDPLUS,788.79999,788.79999,774.70001,781.04999,140887.0
2023-06-13 09:15:00,NSE:MEDPLUS,783.04999,788.90002,770.15002,776.90002,64455.0
2023-06-14 09:15:00,NSE:MEDPLUS,777.90002,785.84998,767.95001,771.45001,58695.0


In [5]:
data = tv.get_hist(symbol = "SBIN", exchange='NSE', interval= Interval.in_daily, n_bars = 1000)

In [6]:
data

,symbol,open,high,low,close,volume
datetime,,,,,,
2019-06-06 09:15:00,NSE:SBIN,349.00000,349.60001,335.89999,336.89999,41941144.0
2019-06-07 09:15:00,NSE:SBIN,337.00000,342.75000,336.00000,342.04999,29739710.0
2019-06-10 09:15:00,NSE:SBIN,346.00000,347.29999,340.14999,344.29999,20500542.0
2019-06-11 09:15:00,NSE:SBIN,345.04999,348.29999,342.85001,347.10001,16281297.0
2019-06-12 09:15:00,NSE:SBIN,346.95001,346.95001,342.29999,344.00000,11564970.0
...,...,...,...,...,...,...
2023-06-12 09:15:00,NSE:SBIN,580.00000,581.50000,577.70001,578.65002,8786066.0
2023-06-13 09:15:00,NSE:SBIN,579.45001,581.40002,574.25000,576.40002,12612098.0
2023-06-14 09:15:00,NSE:SBIN,577.00000,578.54999,575.25000,576.54999,8948696.0
